In [ ]:
import logging
import os
import json
import datetime as dt 
from os.path import join
from logging.handlers import RotatingFileHandler
import pandas as pd
import psycopg2
import pandas as pd
from datetime import datetime, timedelta
import requests
import time
# import ibm_db

#1
json_path = r'C:\Users\00220401626\Desktop\teams\config.json'
teams_webhook_url = "https://kyoceragp.webhook.office.com/webhookb2/32c8e9d8-fee1-47b5-8f88-7c024b169401@82cc187e-25d5-45e4-8c34-8434bf6075fe/IncomingWebhook/c008d92acff64b93a35fa912edecdbf3/329f5970-aad5-4b39-90b5-cf6569c670e1"   # Test          
teams_webhook_urll  = "https://kyoceragp.webhook.office.com/webhookb2/32c8e9d8-fee1-47b5-8f88-7c024b169401@82cc187e-25d5-45e4-8c34-8434bf6075fe/IncomingWebhook/7aae0d10bdf2479da78472f52f6b1ebd/329f5970-aad5-4b39-90b5-cf6569c670e1" # Original

try:
   
    with open(json_path) as json_file:
        config = json.load(json_file)

    # Extract Configuration values
    database_1 = config['database_1']
    host_1 = config['host_1']
    port_1 = config['port_1']
    protocol_1 = config['protocol_1']
    user_1 = config['user_1']
    password_1 = config['password_1']
    
    database_2 = config['database_2']
    host_2 = config['host_2']
    port_2 = config['port_2']
    protocol_2 = config['protocol_2']
    user_2 = config['user_2']
    password_2 = config['password_2']

    log = config['log']  
    days = int(config['days'])
    insertion_time = str(config['insertion_time'])
    inserted_time = dt.datetime.strptime(insertion_time, '%Y-%m-%d %H:%M:%S')
    inserted_day = inserted_time.strftime('%Y%m%d')

    # Logging Settings    
    if not os.path.exists(log):
        os.makedirs(log, exist_ok=True)
        
    dt_st = datetime.now()
    log_path = join(log, f"debug_{dt_st.strftime('%Y%m%d')}.log")
    log_formatter = logging.Formatter('%(asctime)s [%(levelname)s]: %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
    log_handler = RotatingFileHandler(filename=log_path, maxBytes=1048576, backupCount=10, delay=True)
    log_handler.setFormatter(log_formatter)
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)
    logger.addHandler(log_handler)

    print("Started")
    logging.info("Started")
  
    logging.info("Step_1: Extract JSON Configuration Values")
    print("Step_1: Extract JSON Configuration Values")

except Exception as e:
    logging.error(f"Step_1: An error occurred while loading configuration: {e}")
    print(f"Step_1: An error occurred while loading configuration: {e}")

#2
def take_data_from_database():  
    try:
        conn = psycopg2.connect(database=database_1, user=user_1, password=password_1, host=host_1, port=port_1)
        sql_query = """
                    SELECT *
                    FROM mm_01
                    WHERE state IN (2, 3) AND timedate > %s;
                """
        df = pd.read_sql_query(sql_query, conn, params=(inserted_time,))

        logging.info("Step_2: Data is taken from the Database.")
        print("Step_2: Data is taken from the Database.")

        return df
    
    except Exception as e:
        logging.error(f"Step_2: Error: {e}")
        print(f"Step_2: Error : {e}")
#3 
def insert_into_database(df):
    try:
        dsn = f"DATABASE={database_2};HOSTNAME={host_2};PORT={port_2};PROTOCOL={protocol_2};UID={user_2};PWD={password_2}"
        conn = ibm_db.connect(dsn, "", "")
        if conn:
            logging.info("Step_3: 1.Connected to Database")
            print("Step_3: 1.Connected to Database")
    except Exception as e:
        print("Step_3: 1.Failed to Connect to Database:", e)
        logging.error("Step_3: 1.Failed to Connect to Database:", e)


    values = ''
    for index, row in df.iterrows():
        value_1 = 'Particles'
        value_2 = 'MM_01'
        value_3 = row['timedate']
        value_4 = row['state']
        value_5 = '0.5, 1.0, 5.0, 10.0'
        value_6 = f"{row['0.5']}, {row['1.0']}, {row['5.0']}, {row['10.0']}"

        values += f"('{value_1}','{value_2}', '{value_3}', '{value_4}', '{value_5}', '{value_6}'),"
        
    values = values[:-1] 

    insert_query = f"INSERT INTO IOT_DATA.THRESHOLD_ALERT(CATEGORY, EQP, TS_DT, STATE, DATA_COLUMN, DATA_VALUE) VALUES {values};"
    stmt = ibm_db.prepare(conn, insert_query)

    try:
        if ibm_db.execute(stmt):
            logging.info("Step_3: 2.Inserted to Database")
            print("Step_3: 2.Inserted to Database")

    except Exception as e:
        logging.error(f"Step_3: 2.An error occurred while inserting into the database")
        print(f"Step_3: 2.Error: {e}")
#4
def send_teams_message(df):
    df['state'] = df['state'].replace({2: '警報', 3: '警告'})

    message_text = ""
    counter = 0

    for index, row in df.iterrows():
        value_0 = row['timedate']
        value_1 = row['state']  
        value_2 = '気中パーティクル'
        value_3 = 'mm_01'

        value_4 = row['0.5']
        value_5 = row['1.0']  
        value_6 = row['5.0'] 
        value_7 = row['10.0']
            
        link = "http://mb.in.kyocera.co.jp/motionboard/main?mbid=fidkma77jxaifa65c65aaahv5mwqy"

        message_text += f"超過アラート   : {value_0}\n\n判定: {value_1}\n\n種別: {value_2}\n\n装置: {value_3}\n\nデータ:\n\n\t 0.5  = {value_4}\n\t 1.0  = {value_5}\n\t 5.0  = {value_6}\n\t 10.0 = {value_7}\nしきい値: \n\n↓MotionBoardへのリンク\n\n{link}\n\n----------------------------\n\n"

        counter += 1

        if counter == 5:
            payload = {"text": message_text}
            response = requests.post(teams_webhook_url, data=json.dumps(payload), verify=False)
            message_text = ""
            counter = 0

            if response.status_code == 200:
                logging.info("Step_4: Message sent successfully to Microsoft Teams")
                print("Step_4: Message sent successfully to Microsoft Teams")      
            else:
                logging.info(f"Step_4: Failed to send message. Status code: {response.status_code}, Response: {response.text}")
                print(f"Step_4: Failed to send message. Status code: {response.status_code}, Response: {response.text}")


    if message_text:
        payload = {"text": message_text}
        response = requests.post(teams_webhook_url, data=json.dumps(payload), verify=False)

        if response.status_code == 200:
            logging.info("Step_4: Message sent successfully to Microsoft Teams")
            print("Step_4: Message sent successfully to Microsoft Teams")      
        else:
            logging.info(f"Step_4: Failed to send message. Status code: {response.status_code}, Response: {response.text}")
            print(f"Step_4: Failed to send message. Status code: {response.status_code}, Response: {response.text}")
    else:
        logging.info(f"Step_4: Failed to send message. Message is empty.")
        print(f"Step_4: Failed to send message. Message is empty.")

#5
def update_json(df):
    try:
        if df is not None and not df.empty:
            config['insertion_time'] = str(df['timedate'].max())
            time=config['insertion_time']
            json_data = json.dumps(config, indent=2)

            with open(json_path, 'w') as output_file:
                output_file.write(json_data)

            logging.info(f"Step_5: JSON Insertion Time is updated to {time}")
            print(f"Step_5: JSON Insertion Time is updated to  {time}")

        else:
            logging.info(f"Step_5: There is no time data")
            print(f"Step_5: There is no time data")
            

    except Exception as e:
        logging.error(f"Step_5: Error: {e}")
        print(f"Step_5: Error : {e}")

#6 Delete Log files if created day period passed 30 days                          
def delete_old_logs(log, days):
    period_time = datetime.now() - timedelta(days=days)

    for filename in os.listdir(log):
        if filename.endswith(".log"):
            filepath = os.path.join(log, filename)
            try:
                if os.path.isfile(filepath) and datetime.fromtimestamp(os.path.getctime(filepath)) < period_time:
                    os.remove(filepath)
                    logging.info(f"Deleted log file: {filepath}")
                    print(f"Deleted log file: {filepath}")
            except Exception as e:
                print(f"Error deleting file {filepath}: {e}")
                logging.info(f"Error deleting file {filepath}: {e}")

start_time = time.time()

df = take_data_from_database() 
# insert_into_database(df)
send_teams_message(df)
update_json(df)
delete_old_logs(log, days)
end_time = time.time()

total_time = end_time - start_time
print("Finished. Total spent time: ", total_time)
logging.info("Finished. Total spent time: %s\n", total_time)


In [1]:
import logging
import os
import json
import datetime as dt 
from os.path import join
from logging.handlers import RotatingFileHandler
import pandas as pd
import psycopg2
import pandas as pd
from datetime import datetime, timedelta
import requests
import time
from pymsteams import connectorcard

# import ibm_db

#1
json_path = r'C:\Users\00220401626\Desktop\teams\config.json'
teams_webhook_url = "https://kyoceragp.webhook.office.com/webhookb2/32c8e9d8-fee1-47b5-8f88-7c024b169401@82cc187e-25d5-45e4-8c34-8434bf6075fe/IncomingWebhook/c008d92acff64b93a35fa912edecdbf3/329f5970-aad5-4b39-90b5-cf6569c670e1"   # Test          
teams_webhook_urll  = "https://kyoceragp.webhook.office.com/webhookb2/32c8e9d8-fee1-47b5-8f88-7c024b169401@82cc187e-25d5-45e4-8c34-8434bf6075fe/IncomingWebhook/7aae0d10bdf2479da78472f52f6b1ebd/329f5970-aad5-4b39-90b5-cf6569c670e1" # Original

try:
   
    with open(json_path) as json_file:
        config = json.load(json_file)

    # Extract Configuration values
    database_1 = config['database_1']
    host_1 = config['host_1']
    port_1 = config['port_1']
    protocol_1 = config['protocol_1']
    user_1 = config['user_1']
    password_1 = config['password_1']
    
    database_2 = config['database_2']
    host_2 = config['host_2']
    port_2 = config['port_2']
    protocol_2 = config['protocol_2']
    user_2 = config['user_2']
    password_2 = config['password_2']

    log = config['log']  
    days = int(config['days'])
    insertion_time = str(config['insertion_time'])
    inserted_time = dt.datetime.strptime(insertion_time, '%Y-%m-%d %H:%M:%S')
    inserted_day = inserted_time.strftime('%Y%m%d')

    # Logging Settings    
    if not os.path.exists(log):
        os.makedirs(log, exist_ok=True)
        
    dt_st = datetime.now()
    log_path = join(log, f"debug_{dt_st.strftime('%Y%m%d')}.log")
    log_formatter = logging.Formatter('%(asctime)s [%(levelname)s]: %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
    log_handler = RotatingFileHandler(filename=log_path, maxBytes=1048576, backupCount=10, delay=True)
    log_handler.setFormatter(log_formatter)
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)
    logger.addHandler(log_handler)

    print("Started")
    logging.info("Started")
  
    logging.info("Step_1: Extract JSON Configuration Values")
    print("Step_1: Extract JSON Configuration Values")

except Exception as e:
    logging.error(f"Step_1: An error occurred while loading configuration: {e}")
    print(f"Step_1: An error occurred while loading configuration: {e}")

#2
def take_data_from_database():  
    try:
        conn = psycopg2.connect(database=database_1, user=user_1, password=password_1, host=host_1, port=port_1)
        sql_query = """
                    SELECT *
                    FROM mm_01
                    WHERE state IN (2, 3) AND timedate > %s;
                """
        df = pd.read_sql_query(sql_query, conn, params=(inserted_time,))

        logging.info("Step_2: Data is taken from the Database.")
        print("Step_2: Data is taken from the Database.")

        return df
    
    except Exception as e:
        logging.error(f"Step_2: Error: {e}")
        print(f"Step_2: Error : {e}")


#3

class MsTeams:
    def __init__(self, webhook):
        self.teams = connectorcard(webhook)

    def create_mention_payload(self, user_id, user_display_name):
        mention_entity = {
            "type": "mention",
            "text": f"<at>{user_display_name}</at>",
            "mentioned": {
                "id": user_id,
                "name": user_display_name
            }
        }
        adaptive_card_content = {
            "type": "AdaptiveCard",
            "body": [{"type": "TextBlock", "text": f"{mention_entity['text']}"}],
            "$schema": "http://adaptivecards.io/schemas/adaptive-card.json",
            "version": "1.0",
            "msteams": {"entities": [mention_entity]}
        }
        self.teams.payload = {
            "type": "message",
            "attachments": [{
                "contentType": "application/vnd.microsoft.card.adaptive",
                "content": adaptive_card_content
            }]
        }

    def send(self):
        self.teams.send()

user_id = "murodiljon.mirzakhmedov.ke@kyocera.jp"
user_display_name = "Murodiljon Mirzakhmedov　( ﾐﾙｻﾞﾌﾒﾄﾞﾌ ﾑﾛﾃﾞｨﾙｼﾞｮﾝ )"
teams = MsTeams("https://kyoceragp.webhook.office.com/webhookb2/32c8e9d8-fee1-47b5-8f88-7c024b169401@82cc187e-25d5-45e4-8c34-8434bf6075fe/IncomingWebhook/c008d92acff64b93a35fa912edecdbf3/329f5970-aad5-4b39-90b5-cf6569c670e1")
teams.create_mention_payload(user_id, user_display_name)

def send_teams_message(df):
    df['state'] = df['state'].replace({2: '警報', 3: '警告'})

    message_text = ""
    counter = 0

    for index, row in df.iterrows():
        value_0 = row['timedate']
        value_1 = row['state']  
        value_2 = '気中パーティクル'
        value_3 = 'mm_01'

        value_4 = row['0.5']
        value_5 = row['1.0']  
        value_6 = row['5.0'] 
        value_7 = row['10.0']
            
        link = "http://mb.in.kyocera.co.jp/motionboard/main?mbid=fidkma77jxaifa65c65aaahv5mwqy"

        message_text += f"超過アラート   : {value_0}\n\n判定: {value_1}\n\n種別: {value_2}\n\n装置: {value_3}\n\nデータ:\n\n\t 0.5  = {value_4}\n\t 1.0  = {value_5}\n\t 5.0  = {value_6}\n\t 10.0 = {value_7}\nしきい値: \n\n↓MotionBoardへのリンク\n\n{link}\n\n----------------------------\n\n"

        counter += 1

        if counter == 5:
           
            teams.send()
            payload = {"text": message_text}
            response = requests.post(teams_webhook_url, data=json.dumps(payload), verify=False)
            message_text = ""
            counter = 0

            if response.status_code == 200:
                logging.info("Step_4: Message sent successfully to Microsoft Teams")
                print("Step_4: Message sent successfully to Microsoft Teams")      
            else:
                logging.info(f"Step_4: Failed to send message. Status code: {response.status_code}, Response: {response.text}")
                print(f"Step_4: Failed to send message. Status code: {response.status_code}, Response: {response.text}")


    if message_text:
       
        teams.send()
        payload = {"text": message_text}
        response = requests.post(teams_webhook_url, data=json.dumps(payload), verify=False)

        if response.status_code == 200:
            logging.info("Step_4: Message sent successfully to Microsoft Teams")
            print("Step_4: Message sent successfully to Microsoft Teams")      
        else:
            logging.info(f"Step_4: Failed to send message. Status code: {response.status_code}, Response: {response.text}")
            print(f"Step_4: Failed to send message. Status code: {response.status_code}, Response: {response.text}")
    else:
        logging.info(f"Step_4: Failed to send message. Message is empty.")
        print(f"Step_4: Failed to send message. Message is empty.")



start_time = time.time()

df = take_data_from_database() 
send_teams_message(df)

end_time = time.time()

total_time = end_time - start_time
print("Finished. Total spent time: ", total_time)
logging.info("Finished. Total spent time: %s\n", total_time)


Started
Step_1: Extract JSON Configuration Values
Step_2: Error : connection to server at "10.143.64.54", port 5432 failed: Connection timed out (0x0000274C/10060)
	Is the server running on that host and accepting TCP/IP connections?



TypeError: 'NoneType' object is not subscriptable

In [ ]:
import requests
import pandas as pd
from pathlib import Path
import json
import os

mention_excel = "25_Teamsメンションリスト.xlsx"
message_t = "なにかの文字列"
# teams_webhook = "https://kyoceragp.webhook**********（上記で設定したWebhookのURL）"

mention_file = list(Path(r"./file").glob(mention_excel))[0]
df1 = pd.read_excel(mention_file)
display("■メンションリスト", df1)



# アダプティブカードに組み込むメンション用文字列の作成
for i in range(0, len(df1)):
    mailadd = df1.at[i, "Mail_address"]
    mentionto = df1.at[i, "Name"]
    mn_list = mn_list + f"<at>{mentionto}</at>さん、"
    mnadd_list = mnadd_list + f"""
                        {{
                            "type": "mention",
                            "text": "<at>{mentionto}</at>",
                            "mentioned": {{
                                "id": "{mailadd}",
                                "name": "{mentionto}"
                            }}
                        }},"""
mnadd_list = mnadd_list[:-1]






# アダプティブカードの設定
message2 = f"""
{{
    "type": "message",
    "attachments": [
        {{
            "contentType": "application/vnd.microsoft.card.adaptive",
            "content": {{
                "type": "AdaptiveCard",
                "version": "1.0",
                "body": [
                    {{
                        "type": "TextBlock",
                        "text": " ",
                        "size": "Large",
                        "weight": "Bolder"
                    }},
                    {{
                        "type": "TextBlock",
                        "text": "{mn_list}",
                        "wrap": true
                    }},
                    {{
                        "type": "TextBlock",
                        "text": "{message_t}"
                    }}
                ],
                "actions": [
                    {{
                        "type": "Action.OpenUrl",
                        "url": "http://jnjhp.in.kyocera.co.jp/fw/dfw/DMP/jp/top.html",
                        "title": "人事労務Webのリンクを開く"
                    }}
                ],
                "msteams": {{
                    "width": "Full",
                    "entities": [
                        {mnadd_list}
                    ]
                }}
            }}
        }}
    ]
}}
"""

#         Teamsにメッセージを送る
data2 = json.loads(message2, strict=False)
requests.post(teams_webhook, json.dumps(data2))
print("Python経由でTeamsにメッセージを通知(送信)しました。")

In [ ]:
import logging
import os
import json
import datetime as dt 
from os.path import join
from logging.handlers import RotatingFileHandler
import pandas as pd
import psycopg2
import pandas as pd
from datetime import datetime, timedelta
import requests
import time


# import ibm_db

#1
json_path = r'C:\Users\00220401626\Desktop\teams\config.json'
teams_webhook_url = "https://kyoceragp.webhook.office.com/webhookb2/32c8e9d8-fee1-47b5-8f88-7c024b169401@82cc187e-25d5-45e4-8c34-8434bf6075fe/IncomingWebhook/c008d92acff64b93a35fa912edecdbf3/329f5970-aad5-4b39-90b5-cf6569c670e1"   # Test          
teams_webhook_urll  = "https://kyoceragp.webhook.office.com/webhookb2/32c8e9d8-fee1-47b5-8f88-7c024b169401@82cc187e-25d5-45e4-8c34-8434bf6075fe/IncomingWebhook/7aae0d10bdf2479da78472f52f6b1ebd/329f5970-aad5-4b39-90b5-cf6569c670e1" # Original

try:
   
    with open(json_path) as json_file:
        config = json.load(json_file)

    # Extract Configuration values
    database_1 = config['database_1']
    host_1 = config['host_1']
    port_1 = config['port_1']
    protocol_1 = config['protocol_1']
    user_1 = config['user_1']
    password_1 = config['password_1']
    
    database_2 = config['database_2']
    host_2 = config['host_2']
    port_2 = config['port_2']
    protocol_2 = config['protocol_2']
    user_2 = config['user_2']
    password_2 = config['password_2']

    log = config['log']  
    days = int(config['days'])
    insertion_time = str(config['insertion_time'])
    inserted_time = dt.datetime.strptime(insertion_time, '%Y-%m-%d %H:%M:%S')
    inserted_day = inserted_time.strftime('%Y%m%d')

    # Logging Settings    
    if not os.path.exists(log):
        os.makedirs(log, exist_ok=True)
        
    dt_st = datetime.now()
    log_path = join(log, f"debug_{dt_st.strftime('%Y%m%d')}.log")
    log_formatter = logging.Formatter('%(asctime)s [%(levelname)s]: %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
    log_handler = RotatingFileHandler(filename=log_path, maxBytes=1048576, backupCount=10, delay=True)
    log_handler.setFormatter(log_formatter)
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)
    logger.addHandler(log_handler)

    print("Started")
    logging.info("Started")
  
    logging.info("Step_1: Extract JSON Configuration Values")
    print("Step_1: Extract JSON Configuration Values")

except Exception as e:
    logging.error(f"Step_1: An error occurred while loading configuration: {e}")
    print(f"Step_1: An error occurred while loading configuration: {e}")

#2
def take_data_from_database():  
    try:
        conn = psycopg2.connect(database=database_1, user=user_1, password=password_1, host=host_1, port=port_1)
        sql_query = """
                    SELECT *
                    FROM mm_01
                    WHERE state IN (2, 3) AND timedate > %s;
                """
        df = pd.read_sql_query(sql_query, conn, params=(inserted_time,))

        logging.info("Step_2: Data is taken from the Database.")
        print("Step_2: Data is taken from the Database.")

        return df
    
    except Exception as e:
        logging.error(f"Step_2: Error: {e}")
        print(f"Step_2: Error : {e}")


#3

def send_teams_message(df):
    df['state'] = df['state'].replace({2: '警報', 3: '警告'})

    message_text = ""
    counter = 0

    for index, row in df.iterrows():
        value_0 = row['timedate']
        value_1 = row['state']  
        value_2 = '気中パーティクル'
        value_3 = 'mm_01'

        value_4 = row['0.5']
        value_5 = row['1.0']  
        value_6 = row['5.0'] 
        value_7 = row['10.0']
            
        link = "http://mb.in.kyocera.co.jp/motionboard/main?mbid=fidkma77jxaifa65c65aaahv5mwqy"

        message_text += f"超過アラート   : {value_0}\n\n判定: {value_1}\n\n種別: {value_2}\n\n装置: {value_3}\n\nデータ:\n\n\t 0.5  = {value_4}\n\t 1.0  = {value_5}\n\t 5.0  = {value_6}\n\t 10.0 = {value_7}\nしきい値: \n\n↓MotionBoardへのリンク\n\n{link}\n\n----------------------------\n\n"

        counter += 1

        if counter == 5:
           
            payload = {
                "text": message_text,
                "mentions": [
                    {
                        "mentionType": "person",
                        "mentionText": "murodil.mirzakhmedov@kyocera.jp",
                        "mentioned": {
                            "id": "murodil.mirzakhmedov@kyocera.jp",
                            "name": "Murodiljon Mirzakhmedov　( ﾐﾙｻﾞﾌﾒﾄﾞﾌ ﾑﾛﾃﾞｨﾙｼﾞｮﾝ )"
                        }
                    }
                ]
            }
            response = requests.post(teams_webhook_url, data=json.dumps(payload), verify=False)
            message_text = ""
            counter = 0

            if response.status_code == 200:
                logging.info("Step_4: Message sent successfully to Microsoft Teams")
                print("Step_4: Message sent successfully to Microsoft Teams")      
            else:
                logging.info(f"Step_4: Failed to send message. Status code: {response.status_code}, Response: {response.text}")
                print(f"Step_4: Failed to send message. Status code: {response.status_code}, Response: {response.text}")


    if message_text:
       
        payload = {
            "text": message_text,
            "mentions": [
                {
                    "mentionType": "person",
                    "mentionText": "murodil.mirzakhmedov@kyocera.jp",
                    "mentioned": {
                        "id": "murodil.mirzakhmedov@kyocera.jp",
                        "name": "Murodiljon Mirzakhmedov　( ﾐﾙｻﾞﾌﾒﾄﾞﾌ ﾑﾛﾃﾞｨﾙｼﾞｮﾝ )"
                    }
                }
            ]
        }
        response = requests.post(teams_webhook_url, data=json.dumps(payload), verify=False)

        if response.status_code == 200:
            logging.info("Step_4: Message sent successfully to Microsoft Teams")
            print("Step_4: Message sent successfully to Microsoft Teams")      
        else:
            logging.info(f"Step_4: Failed to send message. Status code: {response.status_code}, Response: {response.text}")
            print(f"Step_4: Failed to send message. Status code: {response.status_code}, Response: {response.text}")
    else:
        logging.info(f"Step_4: Failed to send message. Message is empty.")
        print(f"Step_4: Failed to send message. Message is empty.")



start_time = time.time()

df = take_data_from_database() 
send_teams_message(df)

end_time = time.time()

total_time = end_time - start_time
print("Finished. Total spent time: ", total_time)
logging.info("Finished. Total spent time: %s\n", total_time)
